<a href="https://colab.research.google.com/github/Kvnhooman/AAI520_Final-Project_Group5/blob/tommy-dev-1/Financial_Review_AI_Agents_v_10_7_0651.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Title
**Group 5**

Final Project




___
## Outline


**Core Agent Functions**

* Planning Agent: Plans research steps for stock analysis


* Tool Use Agent: Integrates APIs (Yahoo Finance, SEC EDGAR, News APIs)


* Self-Reflection Agent: Evaluates output quality and iterates


* Learning Agent: Maintains memory across analysis runs


**Workflow Patterns**
* Prompt Chaining: News ingestion → preprocessing → classification → extraction → summarization


* Routing: Directs content to specialist analyzers (earnings, news, market)


* Evaluator-Optimizer: Generates analysis → evaluates quality → refines using feedback

In [2]:
# Install dependencis
# Attribution: Geek for Geeks tutorial - https://www.geeksforgeeks.org/artificial-intelligence/introduction-to-langchain/

# -- CORE LANGCHAIN AND PLUGINS --
!pip install -U langchain langchain-openai langchain-community langchain-google-genai

# -- LARGE MODEL PROVIDERS/SKILL ADAPTERS --
!pip install -U google-generativeai huggingface_hub openai

# -- COMMON TOOLING AND UTILITIES --
!pip install -U python-dotenv yfinance duckduckgo-search

# -- DUCKDUCKGO SEARCH API WRAPPER --
!pip install -U ddgs

# -- OPTIONAL: For advanced memory (semantic search/vector db) --
!pip install -U faiss-cpu  # For in-memory vector DBs (Lightweight)
# If you want persistent/production memory, add chromadb or qdrant-client

# -- Install LangGraph for advanced agent orchestration --
!pip install -U langgraph

# -- (OPTIONAL) For running Python tool actions securely --
!pip install -U restrictedpython

# -- Wikipedia
!pip install wikipedia

# Restart the runtime after running this cell if prompted!

  Using cached openai-2.2.0-py3-none-any.whl.metadata (29 kB)
  Using cached google_ai_generativelanguage-0.6.15-py3-none-any.whl.metadata (5.7 kB)
Using cached google_ai_generativelanguage-0.6.15-py3-none-any.whl (1.3 MB)
Using cached openai-2.2.0-py3-none-any.whl (998 kB)
  Attempting uninstall: openai
    Found existing installation: openai 1.109.1
    Uninstalling openai-1.109.1:
      Successfully uninstalled openai-1.109.1
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.7.0
    Uninstalling google-ai-generativelanguage-0.7.0:
      Successfully uninstalled google-ai-generativelanguage-0.7.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-google-genai 2.1.12 requires google-ai-generativelanguage<1,>=0.7, but you have google-ai-generativelanguage 0.6.15 which is incompatible.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 78.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 59.5 MB/s eta 0:00:00
  Attempting uninstall: lxml
    Found existing installation: lxml 5.4.0
    Uninstalling lxml-5.4.0:
      Successfully uninstalled lxml-5.4.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 15.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=94a57714e645f07

In [16]:
#Import key libraries
import os
import requests
import re

from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import userdata

from langchain_openai import OpenAI, ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts.chat import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import create_react_agent, AgentExecutor, initialize_agent, Tool, AgentType
from langchain_core.tools import tool
from langchain.chains import LLMChain
from langchain.tools import tool  #Newer import for @tool decorator
from langchain.llms import OpenAI

from langgraph.prebuilt import create_react_agent

#Correct import path for YahooFinanceAPI
import yfinance as yf
from langchain_community.tools.yahoo_finance_news import YahooFinanceNewsTool
from langchain.prompts import PromptTemplate

#Memory
from dataclasses import dataclass
from datetime import datetime
from typing import Dict, List, Optional, Any, Union

In [17]:
#Set up LLM API Calls
gemini_key = userdata.get('GEMINI')
hf_key = userdata.get('HF_TOKEN')
openai_key = userdata.get('OPENAI')
fin_news = userdata.get('FIN_API_KEY')
tavily_key = userdata.get('TAVILY_API_KEY')

In [18]:
# Setup Gemini to use in Agents
llm_gemini = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    google_api_key=gemini_key
)

In [4]:
# Test if Gemini LLM is reachable
try:
    response = llm_gemini.invoke("Say 'GoogleAPI-success-check'")
    print("Gemini API Response:", response)
except Exception as e:
    print("Error reaching Gemini API:", e)


Gemini API Response: content='GoogleAPI-success-check' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []} id='run--8360071c-60c1-42cb-b681-687412e9ea08-0'


In [19]:
llm_openai = ChatOpenAI(
    model="gpt-3.5-turbo",  # "gpt-3.5-turbo", "gpt-4-mini", "gpt-5-mini" ...
    openai_api_key=openai_key,  # Your OpenAI API key
    temperature=0.0             # (optional) set as needed
)

In [6]:
#Test OpenAI API
#response = llm_openai.invoke("Explain how neural networks work in 10 words or less.")
response = llm_gemini.invoke("Explain how neural networks work in 10 words or less.")
print(response.content)

Learns patterns from data to make predictions.


## User Input Context Extractor Agent

In [35]:
user_input = """
    "input": (str) Analyze the Apple stock.
    "symbol": (str) AAPL
    "company_name": (str) Apple
    "date": (str) 2025/10/07
    "agent_scratchpad": (str) "",
    "research_notes": (str) ""
    """

In [36]:
def extract_context(user_input):
    """
    Extracts key context fields from user input for agent API use.

    Fields:
      - input (str): raw user text
      - symbol (str): detected uppercase ticker (2-5 chars)
      - company_name (str): placeholder for future NLP extraction
      - date (str): today's date in ISO format (YYYY-MM-DD)
      - agent_scratchpad (str): empty, reserved for intermediate agent notes
      - research_notes (str): empty, reserved for additional notes
    """
    # Extract uppercase ticker symbol (2-5 chars)
    match = re.search(r"\b([A-Z]{2,5})\b", user_input)
    symbol = match.group(1) if match else ""

    # Placeholder for company name extraction
    company_name = ""  # Optional: implement NER for this

    # Today's date in ISO format
    today = datetime.now().date().isoformat()

    # Build context dictionary
    context = {
        'input': user_input,
        'symbol': symbol,
        'company_name': company_name,
        'date': today,
        'agent_scratchpad': '',
        'research_notes': ''
    }
    return context

In [37]:
#Input extraction output
context = extract_context(user_input)
print(context)

{'input': ' \n    "input": (str) Analyze the Apple stock.\n    "symbol": (str) AAPL\n    "company_name": (str) Apple\n    "date": (str) 2025/10/07\n    "agent_scratchpad": (str) "",\n    "research_notes": (str) ""\n    ', 'symbol': 'AAPL', 'company_name': '', 'date': '2025-10-07', 'agent_scratchpad': '', 'research_notes': ''}


## Tool Agent
Integrates APIs (Yahoo Finance, SEC EDGAR, News APIs)

List of Tools:
- Web Search
- API call - Yahoo Finance
- to be updated...

### Tool Functions


In [ ]:
#For potential future use
#from fredapi import Fred
##fred = Fred(api_key='YOUR_API_KEY')
#data = fred.get_series('SP500')

In [38]:
#Yahoo Finance Tool
def get_stock_price(ticker: str) -> str:
    try:
        price = yf.Ticker(ticker).info['regularMarketPrice']
        return f"{ticker} price is {price}"
    except Exception as e:
        return f"Error fetching price for {ticker}: {e}"

yahoo_api_tool = Tool(
    name="YahooFinanceAPI",
    func=get_stock_price,
    description="Queries Yahoo Finance for stock price and financials"
)

In [39]:
#SEC Filings
def get_sec_filings(ticker: str) -> str:
    cik_url = "https://www.sec.gov/files/company_tickers.json"
    headers = {"User-Agent": "tpool@sandiego.edu"}  # Use your real email!
    cik_resp = requests.get(cik_url, headers=headers)
    print(f"cik_resp: {cik_resp.status_code}, {cik_resp.text[:100]}")

    if cik_resp.status_code != 200:
        return f"SEC.gov rejected our request: {cik_resp.status_code}\n{cik_resp.text[:200]}"

    try:
        cik_data = cik_resp.json()
        cik_lookup = {item['ticker']: item['cik_str'] for item in cik_data.values()}
        cik = cik_lookup.get(ticker.upper())
    except Exception as e:
        return f"Error parsing CIK data: {e}"

    if not cik:
        return f"CIK for {ticker} not found."

    filings_url = f"https://data.sec.gov/submissions/CIK{cik:0>10}.json"
    filings_resp = requests.get(filings_url, headers=headers)

    try:
        data = filings_resp.json() if filings_resp.status_code == 200 else {}
        filings = data.get('filings', {}).get('recent', {})
        if filings:
            forms = filings.get('form', [])[:3]
            filing_dates = filings.get('filingDate', [])[:3] #If successful, get filing dates
            filing_links = filings.get('primaryDocument', [])[:3] #Also get three primary documents for agent
            result = []
            for f, d, l in zip(forms, filing_dates, filing_links):
                result.append(f"{f} on {d}: {l}")
            return f"Latest filings for {ticker}:\n" + "\n".join(result)
        else:
            return f"No filings found for {ticker}."
    except Exception as e:
        return f"Error loading filings for {ticker}: {e}"


sec_api_tool = Tool(
    name="SECEDGARAPI",
    func=get_sec_filings,
    description="Retrieves SEC filings on stock symbol"
)

In [32]:
#For debugging
#print(get_sec_filings("TSLA"))

In [40]:
def get_fin_news(symbol: str) -> str:
    api_key = fin_news
    url = "https://newsapi.org/v2/everything"
    params = {
        "q": symbol,
        "apiKey": api_key,
        "sortBy": "publishedAt",
        "language": "en"
    }
    response = requests.get(url, params=params)
    if response.status_code != 200:
        return f"API error {response.status_code}: {response.text[:200]}"
    data = response.json()
    articles = data.get("articles", [])
    if not articles:
        return f"No news found for {symbol}. Full message: {data.get('message', '')}"
    return "\n".join([a["description"] or a["title"] for a in articles[:3]])


news_api_tool = Tool(
    name="NewsAPI",
    func=get_fin_news,  #Assumes you've defined this class
    description="Finds recent financial news on stock symbol"
)

In [41]:
def get_fin_news_tavily(symbol: str) -> str:
    api_key = tavily_key
    url = "https://api.tavily.com/search"
    # You can optimize the query for financial news by including keywords:
    query = f"{symbol} financial news"
    payload = {
        "query": query,
        "api_key": api_key,
        "max_results": 3,  # You can set number of results as you prefer
    }
    response = requests.post(url, json=payload)
    if response.status_code == 200:
        data = response.json()
        results = data.get("results", [])
        # Each result contains title, link, snippet, etc.
        return "\n".join([r.get("title", "No title") for r in results]) if results else "No news found."
    else:
        return f"Error from Tavily: {response.status_code} {response.text}"

tavily_news_tool = Tool(
    name="TavilyNewsSearch",
    func=get_fin_news_tavily,
    description="Searches web for recent financial news about a stock symbol using Tavily API."
)

In [42]:
#Wikipedia Search Tool
from wikipedia import summary

def search_wikipedia(query: str) -> str:
    # If query looks like a ticker (e.g., all caps, 2-5 chars), get company name
    if query.isupper() and 2 <= len(query) <= 5:
        company_name = get_company_name(query)
        search_term = f"{company_name} stock"
    else:
        search_term = query

    try:
        return summary(search_term, sentences=2)
    except Exception:
        # Fallback, try just company name (without "stock")
        if search_term != query:
            try:
                return summary(company_name, sentences=2)
            except Exception:
                pass
        return "I couldn't find any information on that."

wikipedia_tool = Tool(
    name="WikipediaSearch",
    func=search_wikipedia,
    description="Searches Wikipedia and returns a summary for a stock symbol or company name."
)

In [43]:
from transformers import pipeline

class HuggingFaceSentimentTool:
    def __init__(self):
        self.classifier = pipeline("sentiment-analysis")

    def analyze(self, text):
        result = self.classifier(text)[0]
        # result['label'] is 'POSITIVE' or 'NEGATIVE'
        return 1 if result['label'] == "POSITIVE" else -1

sentiment_api_tool = Tool (
    name="HuggingFaceSentiment",
    func=HuggingFaceSentimentTool().analyze,
    description="Analyzes sentiment of text using HuggingFace"
)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Device set to use cpu


In [44]:
#Analyzer specialist tool
class EarningsAnalyzer:
    def __init__(self, yahoo_tool, sec_tool):
        self.yahoo_tool = yahoo_tool
        self.sec_tool = sec_tool

    def analyze(self, symbol, messages=None):
        #Try to find earnings info in past messages
        yahoo_data = None
        if messages:
            for msg in reversed(messages):
                # ToolMessage expected structure
                # Check by tool name and relevant symbol
                if (
                    hasattr(msg, "name") and msg.name == "YahooFinanceAPI"
                    and hasattr(msg, "content") and symbol in msg.content
                ):
                    yahoo_data = msg.content  # or parse accordingly
                    break
        #If not found, call API
        if yahoo_data is None:
            yahoo_data = self.yahoo_tool.fetch_earnings(symbol)
        #Add in for SEC Data ---------------------------------------------------
        #Build and return your summary
        return f"Analysis based on earnings: {yahoo_data}"

earnings_specialist = Tool(
    name="EarningsAnalyzer",
    func=lambda symbol, messages: EarningsAnalyzer(yahoo_api_tool, sec_api_tool).analyze(symbol, messages=messages),
    description="Expert analysis of earnings via Yahoo Finance and SEC filings"
)

In [45]:
class SentimentAnalyzer:
    def __init__(self, sentiment_tool, news_tool):
        self.sentiment_tool = sentiment_tool  # expects .analyze(text)
        self.news_tool = news_tool            # expects .get_news(symbol)

    def analyze(self, symbol, messages=None):
        # 1. Find any recent news/sentiment in agent messages
        headlines = []
        if messages:
            for msg in reversed(messages):
                # If you have ToolMessage for news and symbol in content
                if hasattr(msg, "name") and msg.name in ["NewsAPI", "TavilyNewsSearch"]:
                    if hasattr(msg, "content") and symbol in msg.content:
                        # Try splitting headlines (change this parsing as needed)
                        headlines += msg.content.split("\n")
                # If you cache sentiment scores/results, you can parse those too!

        # 2. If no headlines found, call news API/tool
        if not headlines:
            news_items = self.news_tool.get_news(symbol)
            headlines = [item['headline'] for item in news_items]

        # 3. Get sentiment scores for each headline
        if headlines:
            scores = [self.sentiment_tool.analyze(h) for h in headlines]
            if scores:
                mean_score = sum(scores) / len(scores)
                recommendation = self._interpret_score(mean_score)
                return (f"Average news sentiment for {symbol}: {mean_score:.2f}\n"
                        f"Recommendation: {recommendation}")
            else:
                return "No sentiment scores could be computed."
        else:
            return "No recent news headlines found."

    def _interpret_score(self, score):
        # Customize thresholds to your model's scoring scale
        if score > 0.2:
            return "Buy"
        elif score > -0.2:
            return "Hold"
        else:
            return "Sell"

sentiment_anaylsis_specialist = Tool(
    name="SentimentAnalyzer",
    func=lambda symbol, messages: SentimentAnalyzer(sentiment_api_tool, news_api_tool).analyze(symbol, messages=messages),
    description="Aggregates news sentiment for a stock and suggests buy/hold/sell."
)

In [ ]:
#simpler way to create tool?
#def get_weather(city: str) -> str:
#    """Get weather for a given city."""
#    return f"It's always sunny in {city}!"

In [46]:
#Create the tools list
tools = [
    yahoo_api_tool,
    sec_api_tool,
    news_api_tool,
    tavily_news_tool,
    wikipedia_tool,
    sentiment_api_tool,
    earnings_specialist,
    sentiment_anaylsis_specialist,
    wikipedia_tool
]

### Tools Agent Function

In [76]:
THOROUGH_ANALYSIS_PROMPT = """
You are a comprehensive financial research agent. You MUST use multiple tools to conduct deep analysis.

MANDATORY RESEARCH STEPS - Complete ALL of these:

1. **Company Overview**: Get basic company information, sector, and key metrics
2. **Financial Data**: Retrieve latest quarterly/annual financials (revenue, profit, cash flow)
3. **SEC Filings**: Download and analyze recent 10-K and 10-Q filings for context
4. **Recent News**: Gather latest news, earnings reports, and analyst coverage
5. **Market Context**: Check broader market conditions, sector performance
6. **Peer Comparison**: Compare key metrics against 2-3 competitors
7. **Technical Analysis**: Get recent price action, trends, and key levels
8. **Analyst Sentiment**: Collect analyst ratings, price targets, and recommendations

TOOL USAGE REQUIREMENTS:
- Use at least 4 different tools for each analysis
- Cross-reference information from multiple sources
- If a tool fails, try alternative tools for the same data
- Always explain your reasoning between tool calls

STOPPING RULE: Once you have basic financials, recent news, and market context, conclude your analysis. Do not seek additional tools or data.

**Your goal** is a concise investment overview, not exhaustive research.

For {symbol}, provide a comprehensive investment analysis covering all aspects above.
Be thorough - this analysis will inform major investment decisions.
"""

tools_agent = create_react_agent(
    model=llm_openai,
    tools=tools,
    prompt=THOROUGH_ANALYSIS_PROMPT,
    debug=True
)

In [ ]:
#For testing
#Example:
#tools_output = tools_agent.invoke({"messages": [{'role': 'user', 'content': 'Research TSLA'}]})
#print(result)

## Self Reflection & Evaluation Agent
Evaluates output quality

In [69]:
# Define Self Evaluation agent

EVAL_PROMPT = """
You are an expert evaluator. Your primary job is to give feedback on the analysis below, NOT to overwrite or revise it.

Instructions:

- Always display the full analysis/summary input *exactly as received* at the start of your answer, clearly labeled.
- Provide your commentary (improvement, completeness, feedback) **separately after the full input**.
- If human feedback is supplied, include your response to it at the end **without changing the original summary**.

FORMAT STRICTLY LIKE THIS:
---
Original Analysis:
{input}

Evaluator Commentary:
[Your bullet points: Completeness, Succinctness, Accuracy, Clarity, Human Feedback summary, Suggestions, etc.]

---

Never rewrite or summarize the original analysis. Only provide clear, constructive evaluator commentary after reproducing the input in its original form.

--- Human Feedback ---
{human_feedback}
"""

evaluator_agent = create_react_agent(
    model=llm_openai,
    tools = [],
    prompt=EVAL_PROMPT
)

## Optimization Loop

In [73]:
import textwrap

def print_wrapped(text, width=80):
    for line in text.splitlines():
        print(textwrap.fill(line, width=width))

In [74]:
def optimization_loop(tools_agent, evaluator_agent, user_input):
    # Step 1: Run tools agent
    print("Conducting research...")
    tools_output = tools_agent.invoke(user_input)
    print("\n--- Analysis Summary ---")
    #print(tools_output)
    print_wrapped(str(tools_output))

    # Step 2: Ask user for feedback
    human_feedback = input("\nPlease enter your feedback (areas to improve, missing info, corrections):\n")

    # Step 3: Evaluate and revise summary using feedback
    eval_payload = {
        "input": str(tools_output),
        "human_feedback": human_feedback
    }
    print("\nRunning evaluator with feedback...")
    revised_output = evaluator_agent.invoke(eval_payload)
    print("\n--- Revised Summary ---")
    #print(revised_output)
    print_wrapped(str(revised_output))

    # Optional: Loop for more feedback
    while True:
        more = input("\nWould you like to refine further? (y/n): ")
        if more.lower().startswith("y"):
            human_feedback = input("Enter any further feedback:\n")
            eval_payload["human_feedback"] = human_feedback
            revised_output = evaluator_agent.invoke(eval_payload)
            print("\n--- Revised Summary ---")
            print(revised_output)
        else:
            break

    print("\nWorkflow complete. Final output above.")
    return revised_output  #Return for pretty print


In [71]:
def print_final_analysis(agent_output, title="Final Analysis Summary"):
    """
    Pretty-prints agent outputs including markdown, string, dict, or LangChain message formats.
    Displays a title, renders bullet points and headings, and handles line breaks gracefully.
    """
    print("\n" + "="*60)
    print(f"{title}")
    print("="*60)

    # Helper to render markdown-style output for terminal
    def render_markdown(md):
        # Render headings
        md = re.sub(r"^### (.+)$", r"\n=== \1 ===\n", md, flags=re.MULTILINE)
        md = re.sub(r"^## (.+)$", r"\n== \1 ==\n", md, flags=re.MULTILINE)
        md = re.sub(r"^# (.+)$", r"\n= \1 =\n", md, flags=re.MULTILINE)
        # Replace double newlines with single blank line for separation
        md = re.sub(r"\n{3,}", "\n\n", md)
        # Print with blank lines between paragraphs/bullets
        for para in md.split('\n\n'):
            print(para.strip())
            print()

    # Handle string output
    if isinstance(agent_output, str):
        render_markdown(agent_output)
        return

    # Handle dict outputs (common in LangChain)
    if isinstance(agent_output, dict):
        # Look for 'messages' (LangChain) or 'output'
        if "messages" in agent_output:
            messages = agent_output["messages"]
            for idx, msg in enumerate(messages):
                content = getattr(msg, "content", str(msg))
                if len(messages) > 1:
                    print(f"Message {idx+1}:")
                render_markdown(content)
        elif "output" in agent_output and isinstance(agent_output["output"], str):
            render_markdown(agent_output["output"])
        else:
            # Generic dict pretty-print
            for k, v in agent_output.items():
                print(f"{k}:")
                render_markdown(str(v))
        return

    # Handle LangChain AIMessage or other objects with .content
    content = getattr(agent_output, "content", None)
    if content:
        render_markdown(content)
        return

    # Fallback
    print(agent_output)


## Learning/Memory Agent
Maintains memory across analysis runs

In [54]:
#MEMORY AGENT CELL 1
#Session-scoped memory

@dataclass
class MemoryItem:
    symbol: str
    question: str
    answer: str
    created_at: str
    meta: Dict[str, Any]

class SessionMemory:
    def __init__(self, max_items: int = 200, max_per_symbol: int = 10):
        self._store: Dict[str, List[MemoryItem]] = {}
        self.max_items = max_items
        self.max_per_symbol = max_per_symbol

    def remember(self, symbol: str, question: str, answer: str, **meta) -> None:
        symbol = (symbol or "GENERIC").upper().strip()
        item = MemoryItem(
            symbol=symbol,
            question=(question or "").strip(),
            answer=(answer or "").strip(),
            created_at=datetime.utcnow().isoformat(timespec="seconds"),
            meta=meta or {}
        )
        bucket = self._store.setdefault(symbol, [])
        bucket.append(item)
        if len(bucket) > self.max_per_symbol:
            del bucket[0 : len(bucket) - self.max_per_symbol]
        self._cap_global()

    def recall(self, symbol: str, question: Optional[str] = None) -> Optional[str]:
        symbol = (symbol or "GENERIC").upper().strip()
        bucket = self._store.get(symbol, [])
        if not bucket:
            return None
        if not question:
            return bucket[-1].answer
        q = (question or "").strip()
        for item in reversed(bucket):
            if item.question == q:
                return item.answer
        return None

    def latest(self, symbol: str) -> Optional[MemoryItem]:
        symbol = (symbol or "GENERIC").upper().strip()
        bucket = self._store.get(symbol, [])
        return bucket[-1] if bucket else None

    def _cap_global(self):
        all_items = []
        for sym, bucket in self._store.items():
            for it in bucket:
                all_items.append((it.created_at, sym, it))
        if len(all_items) <= self.max_items:
            return
        all_items.sort(key=lambda x: x[0])  # oldest first
        to_drop = len(all_items) - self.max_items
        cutoff = set(id(it) for _, _, it in all_items[:to_drop])
        for sym in list(self._store.keys()):
            self._store[sym] = [it for it in self._store[sym] if id(it) not in cutoff]

SESSION_MEMORY = SessionMemory()

def extract_symbol(text: str) -> str:
    """
    Grab a likely ticker from the user_input like 'Analyze the SPY stock ticker'.
    Simple heuristic: first ALL-CAPS token 1-5 chars (e.g., AAPL, MSFT, SPY).
    Falls back to 'GENERIC' if none found.
    """
    if not text:
        return "GENERIC"
    candidates = re.findall(r"\b[A-Z]{1,5}\b", text)
    # Light filter for common English words
    stop = {"THE","AND","FOR","WITH","FROM","THIS","THAT","YOUR","HAVE","HOLD"}
    for c in candidates:
        if c not in stop:
            return c
    return "GENERIC"

def as_text(x: Any) -> str:
    """
    Normalize whatever comes back from planner/tools/evaluator/optimizer into a string.
    Works with LangChain AgentExecutor outputs (dict), AIMessage, or raw str.
    """
    try:
        # AIMessage / ChatMessage
        if hasattr(x, "content"):
            return str(x.content)
        # Agent-like dicts
        if isinstance(x, dict):
            if "output" in x and isinstance(x["output"], str):
                return x["output"]
            if "messages" in x and isinstance(x["messages"], list):
                return "\n\n".join(
                    (m.content if hasattr(m, "content") else str(m))
                    for m in x["messages"]
                )
        # plain string
        if isinstance(x, str):
            return x
        return str(x)
    except Exception:
        return str(x)

## Multiple Agent Setup

In [75]:
# Example usage
final_analysis = optimization_loop(tools_agent, evaluator_agent, context)

Running analysis...
[updates] {'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_RZhMtaeh1zti3IlzHea4JBEc', 'function': {'arguments': '{"__arg1": "AAPL"}', 'name': 'YahooFinanceAPI'}, 'type': 'function'}, {'id': 'call_I5JPhdJ17J9STMsaZuFq6VCf', 'function': {'arguments': '{"__arg1": "AAPL"}', 'name': 'SECEDGARAPI'}, 'type': 'function'}, {'id': 'call_XvcdBnrGIthqPndIEvK2Pvzc', 'function': {'arguments': '{"__arg1": "AAPL"}', 'name': 'NewsAPI'}, 'type': 'function'}, {'id': 'call_tungj9rUcqR82ikSD5eooDBJ', 'function': {'arguments': '{"__arg1": "AAPL"}', 'name': 'WikipediaSearch'}, 'type': 'function'}, {'id': 'call_l2nlb2jhpthcpFUKaZS5kfRf', 'function': {'arguments': '{"__arg1": "AAPL"}', 'name': 'SentimentAnalyzer'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 109, 'prompt_tokens': 555, 'total_tokens': 664, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning

GraphRecursionError: Recursion limit of 25 reached without hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/GRAPH_RECURSION_LIMIT

In [64]:
print_final_analysis(final_analysis)


Final Analysis Summary
=== Evaluation of Analysis ===

The analysis provided a comprehensive overview of the topic, covering all important steps and points. It was succinct and accurate, supported by real information. The clarity of the analysis was well-maintained through organized content.

=== Suggestions for Improvement ===

The analysis could benefit from incorporating specific examples or case studies to further illustrate key points. Additionally, providing more context or background information at the beginning could enhance the reader's understanding.

Overall, the analysis is adequate.



In [66]:
# After running the user query through your whole pipeline:
user_question = user_input
symbol = extract_symbol(user_input)
final_answer = as_text(final_analysis)   # Use your utility function

SESSION_MEMORY.remember(symbol, user_question, final_answer)

# Later, you can recall the latest answer for "AAPL":
prev = SESSION_MEMORY.recall("AAPL")
if prev:
    print("Previous answer for AAPL:", prev)

Previous answer for AAPL: ### Evaluation of Analysis

The analysis provided a comprehensive overview of the topic, covering all important steps and points. It was succinct and accurate, supported by real information. The clarity of the analysis was well-maintained through organized content. 

### Suggestions for Improvement

The analysis could benefit from incorporating specific examples or case studies to further illustrate key points. Additionally, providing more context or background information at the beginning could enhance the reader's understanding. 

Overall, the analysis is adequate.


/tmp/ipython-input-3075633432.py:24: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  created_at=datetime.utcnow().isoformat(timespec="seconds"),
